# Kaggle Competition - LinearSVC

In [61]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import urllib.request

from IPython.core.interactiveshell import InteractiveShell
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsOneClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [62]:
# load data
x = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/train_features'))
test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
y = np.array(pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/train_labels')))
y = y.astype(float)

# load transformed image data
color_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/color_features'))
compress_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/compress_features'))
crop_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/crop_features'))
crop_to_corner_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/crop_to_corner_features'))
homography_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/homography_features'))
mirror_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/mirror_features'))
rotate30_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/rotate30_features'))
scale_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/scale_features'))

# subset the data (if wanting to use a smaller number of classes)
classes = np.unique(y)
index = np.ravel(np.nonzero(np.in1d(y, classes)))
x_subset = x[index]
y_subset = y[index]

In [63]:
def computeobj(beta, lamb, x, y):
    # number of observations
    n = x.shape[0]
    
    # compute objective function
    obj = (1/n)*np.sum(np.maximum(0, 1 - y*np.dot(x, beta))**2) + lamb*np.sum(beta**2)
    
    return obj


def computegrad(beta, lamb, x, y):
    # number of observations
    n = x.shape[0]
    
    # compute gradient of objective function
    grad_beta = -(2/n)*np.dot(x.T, y*np.maximum(0, 1 - y*np.dot(x, beta))) + 2*lamb*beta
    
    return grad_beta


def backtracking(beta, lamb, x, y, eta=1, alpha=0.5, gamma=0.8, max_iter=100):
    # initialize variables
    grad_beta = computegrad(beta, lamb, x, y)
    norm_grad_beta = np.sqrt(np.sum(grad_beta**2))
    found_eta = 0
    t = 0
    
    # loop through until eta found or max iterations reached
    while found_eta == 0 and t < max_iter:
        if (computeobj(beta - eta*grad_beta, lamb, x, y) <
                computeobj(beta, lamb, x, y) - alpha*eta*norm_grad_beta**2):
            found_eta = 1
        elif t == max_iter:
            break
        else:
            eta = eta*gamma
            t += 1
    
    return eta


def mylinearsvm(beta_init, theta_init, lamb, x, y, max_iter, eps):
    # initialize variables
    beta = beta_init
    theta = theta_init
    grad_theta = computegrad(theta, lamb, x, y)
    eta_init = 1/(max(np.linalg.eigh(np.dot((1/n)*x.T, x))[0]) + lamb)
    beta_vals = [beta_init]
    t = 0
    
    # loop through until EITHER max iterations reached or threshold of epsilon reached
    while t < max_iter and np.linalg.norm(grad_theta) >  eps:
        eta = backtracking(beta, lamb, x, y, eta=eta_init)
        beta_next = theta - eta*grad_theta
        theta = beta_next + t*(beta_next - beta)/(t + 3)
        grad_theta = computegrad(theta, lamb, x, y)
        beta = beta_next
        beta_vals.append(beta)
        t += 1
        
    return beta_vals


def split_data_equal(x, y, test_set, train_size=0.75):
    # split into train and test sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0, stratify=y)
    
    # center and standardize x values
    x_scaler = StandardScaler().fit(x_train)
    x_train = x_scaler.transform(x_train)
    x_test = x_scaler.transform(x_test)
    test_set = x_scaler.transform(test_set)
    
    return x_train, x_test, y_train, y_test, test_set


def train_alg(x, y, classes, lamb_list):
    # initialize betas and create list for final beta values
    d = x_train.shape[1]
    beta_init = np.zeros(d)
    theta_init = np.zeros(d)
    final_betas = []
    y_values = []

    # loop through each label and perform ovr appending the final betas for each class
    for i in range(len(classes)):
        lamb = lamb_list[i]
        y_binary = copy.deepcopy(y)
        y_binary[y != classes[i]] = -1
        y_binary[y == classes[i]] = 1
        betas = mylinearsvm(beta_init=beta_init, theta_init=theta_init, lamb=lamb, x=x_train, y=y_binary, max_iter=1000, eps=1e-5)[-1]
        final_betas.append(betas)
        
    return np.array(final_betas)


def predict(x, betas, classes):
    # initialize calculated y values
    y_values = []
    
    # loop through set of final betas and calculate y values
    for i in range(len(betas)):
        y_values.append(np.dot(x, betas[i]))
    
    # calculate predicted values
    y_predict = classes[np.argmax(np.array(y_values), axis=0)]
    
    return y_predict


def accuracy_misclass_error(predict, actual):
    # calculate misclassification error
    misclass_error = np.mean(predict != actual)*100
    accuracy = 100 - misclass_error
    
    return accuracy, misclass_error


def display_confusion_matrix(predict, actual):
    # calculate confusion matrix
    conf_mat = confusion_matrix(y_true=actual, y_pred=predict)
    
    # build visual plot
    plt.matshow(conf_mat);
    plt.title('Confusion Matrix');
    plt.xlabel('Predicted Label');
    plt.ylabel('True Label');
    plt.xticks(range(len(classes)), classes);
    plt.yticks(range(len(classes)), classes);
    
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 18
    fig_size[1] = 12
    plt.rcParams["figure.figsize"] = fig_size
    
    return conf_mat
    

def class_diff(classes, conf_matrix):
    # initialize variable to append each individual class percent
    percent_correct = []
    
    # loop through confusion matrix by true label
    for i in range(len(conf_matrix[0, :])):
        class_count = np.sum(conf_matrix[i])
        misclass_count = 0
        
        # loop through confusion matrix by predict label and append percent correct
        for j in range(len(conf_matrix[:, 0])):
            if i != j:
                misclass_count += conf_matrix[i][j]
            else:
                pass
        percent_correct.append(misclass_count/class_count)
        
    # calcuate ordered list of multi-class misclassification error
    ordered_class_diff = np.vstack((classes, np.array(percent_correct))).T
    ordered_class_diff = ordered_class_diff[ordered_class_diff[:, 1].argsort()[::-1]]
    
    return ordered_class_diff


def export_csv(y, name):
    y = y.astype(int)
    df = pd.DataFrame({'Id':np.arange(1, y.shape[0] + 1), 'Prediction':y})
    df.to_csv(name, sep=',', index=False)
    

def decomp_PCA(train, test, test_set, explained_var_threshold=0.95):
    pca = PCA().fit(train)

    pca_explained_var_ratio = pca.explained_variance_ratio_

    pca_explained_var = []
    num_component_vectors = 0

    while np.sum(pca_explained_var) < explained_var_threshold:
        pca_explained_var.append(pca_explained_var_ratio[num_component_vectors])
        num_component_vectors += 1
    #print('# Component Vectors: %d    Explained Var: %f' % (num_component_vectors, np.sum(pca_explained_var)))

    pca = PCA(n_components=num_component_vectors).fit(train)
    x_train = pca.transform(train)
    x_test = pca.transform(test)
    test_set = pca.transform(test_set)
    
    return x_train, x_test, test_set

## Running my own linearSVC and comparing to sklearn's LinearSVC

In [156]:
# split data
test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
x_train, x_test, y_train, y_test, test_set = split_data_equal(x_subset, y_subset, test_set)
n = x_train.shape[0]
d = x_train.shape[1]

In [157]:
# run PCA to reduce dimensionality
x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)

In [158]:
lamb_list = np.ones(x_train.shape[1]).tolist()
betas = train_alg(x=x_train, y=y_train, classes=classes, lamb_list=lamb_list)
y_predict = predict(x=x_test, betas=betas, classes=classes)
accuracy, misclass_error = accuracy_misclass_error(predict=y_predict, actual=y_test)

In [159]:
print('Accuracy: %f%%' % (accuracy))
print('Misclassification Error: %f%%' % (misclass_error))

Accuracy: 65.833333%
Misclassification Error: 34.166667%


In [160]:
pd.DataFrame(betas)

,0,1,2,3,4,5,6,7,8,9,...,546,547,548,549,550,551,552,553,554,555
0,0.002136,-0.001269,-0.000235,0.000293,-0.001169,0.001909,-0.001271,-0.000411,-0.000205,0.000099,...,0.001562,0.001887,0.001932,-0.001100,0.001720,0.000947,-0.002164,0.000508,0.001474,-0.001843
1,0.001897,-0.001972,0.000222,0.001131,-0.001775,0.002404,-0.002060,-0.001217,-0.000176,-0.001744,...,-0.001109,0.001307,0.002123,-0.001019,-0.000218,0.000375,-0.000358,-0.000657,0.000796,0.001298
2,0.000404,0.002034,0.001261,0.002527,-0.000025,0.001677,0.002824,0.000412,0.001538,-0.004038,...,-0.002657,0.000991,0.001389,0.001079,-0.001547,-0.002145,0.001725,-0.000323,0.000070,-0.000577
3,0.000183,0.001618,0.000926,0.001327,-0.000507,-0.000162,0.001078,-0.001120,0.002209,-0.003886,...,0.000629,0.000328,0.002375,0.000888,0.000248,-0.000813,-0.001506,0.000659,-0.001613,0.001205
4,-0.000375,0.000580,-0.000911,0.001112,-0.000347,-0.000588,0.000139,-0.001665,-0.000287,-0.001569,...,0.000428,-0.001422,-0.000903,0.001332,0.000926,0.001016,-0.000450,-0.000617,0.001585,0.001180
5,-0.000201,0.001181,0.000821,0.000429,-0.000698,0.000008,0.000104,0.000148,0.001377,-0.001134,...,0.001040,0.001264,-0.000268,0.001276,-0.001281,-0.000931,0.000401,0.001146,0.000209,-0.000554
6,-0.000402,-0.002209,0.001042,0.002773,-0.001270,0.000623,0.004860,-0.006071,0.001039,0.004327,...,-0.000462,0.001849,-0.001180,0.001380,-0.000265,-0.000312,-0.000473,-0.002439,-0.001372,-0.001074
7,-0.000663,0.000659,-0.000141,0.000586,-0.000428,-0.001122,-0.000119,-0.001751,0.002708,-0.003272,...,0.001486,-0.000731,-0.002754,0.000657,0.001402,-0.001636,0.002426,0.000061,-0.003194,0.000169
8,0.001881,0.001320,0.000358,-0.000506,0.000793,0.000733,0.001644,-0.000198,0.002109,-0.000165,...,-0.000676,0.000411,-0.001626,-0.002409,0.001279,0.001186,-0.003061,-0.000432,-0.001381,-0.000933
9,0.000182,0.001609,0.000479,0.001630,-0.001033,-0.000719,0.001831,-0.001932,0.002011,-0.002148,...,0.002581,0.002190,-0.000464,0.000634,0.000390,-0.001194,0.000761,-0.001640,-0.001679,0.000318


In [161]:
linSVC = LinearSVC(C=1/(2*n*1), fit_intercept=False).fit(x_train, y_train)
betas = linSVC.coef_
y_predict = predict(x=x_test, betas=betas, classes=classes)
accuracy, misclass_error = accuracy_misclass_error(predict=y_predict, actual=y_test)

In [162]:
print('Accuracy: %f%%' % (accuracy))
print('Misclassification Error: %f%%' % (misclass_error))

Accuracy: 65.833333%
Misclassification Error: 34.166667%


In [163]:
pd.DataFrame(betas)

,0,1,2,3,4,5,6,7,8,9,...,546,547,548,549,550,551,552,553,554,555
0,0.002136,-0.001269,-0.000235,0.000293,-0.001169,0.001909,-0.001271,-0.000411,-0.000205,0.000099,...,0.001562,0.001886,0.001932,-0.001100,0.001719,0.000947,-0.002164,0.000508,0.001474,-0.001843
1,0.001897,-0.001972,0.000222,0.001131,-0.001775,0.002404,-0.002060,-0.001217,-0.000176,-0.001744,...,-0.001109,0.001307,0.002123,-0.001019,-0.000218,0.000376,-0.000358,-0.000657,0.000796,0.001298
2,0.000404,0.002034,0.001261,0.002527,-0.000025,0.001677,0.002824,0.000412,0.001538,-0.004038,...,-0.002657,0.000991,0.001389,0.001079,-0.001547,-0.002145,0.001725,-0.000323,0.000070,-0.000577
3,0.000183,0.001618,0.000926,0.001327,-0.000507,-0.000162,0.001078,-0.001120,0.002209,-0.003886,...,0.000629,0.000328,0.002376,0.000888,0.000248,-0.000813,-0.001506,0.000659,-0.001613,0.001205
4,-0.000375,0.000580,-0.000911,0.001112,-0.000347,-0.000588,0.000139,-0.001665,-0.000287,-0.001569,...,0.000428,-0.001422,-0.000902,0.001332,0.000926,0.001016,-0.000450,-0.000617,0.001585,0.001179
5,-0.000201,0.001181,0.000821,0.000429,-0.000698,0.000008,0.000104,0.000148,0.001377,-0.001134,...,0.001041,0.001264,-0.000268,0.001276,-0.001281,-0.000931,0.000401,0.001146,0.000209,-0.000554
6,-0.000402,-0.002209,0.001042,0.002773,-0.001270,0.000623,0.004860,-0.006071,0.001039,0.004327,...,-0.000462,0.001848,-0.001180,0.001379,-0.000265,-0.000312,-0.000473,-0.002439,-0.001371,-0.001074
7,-0.000663,0.000659,-0.000141,0.000586,-0.000428,-0.001122,-0.000119,-0.001751,0.002708,-0.003272,...,0.001486,-0.000731,-0.002754,0.000657,0.001402,-0.001636,0.002426,0.000061,-0.003193,0.000169
8,0.001881,0.001320,0.000358,-0.000506,0.000793,0.000733,0.001644,-0.000198,0.002109,-0.000165,...,-0.000676,0.000411,-0.001625,-0.002409,0.001279,0.001186,-0.003061,-0.000431,-0.001381,-0.000933
9,0.000182,0.001609,0.000479,0.001630,-0.001033,-0.000719,0.001831,-0.001932,0.002011,-0.002148,...,0.002581,0.002190,-0.000464,0.000634,0.000390,-0.001195,0.000762,-0.001640,-0.001679,0.000318


## Running sklearn's LinearSVC for improved performance

In [142]:
# try stacking image tansformations onto the original features
allfeatures = np.vstack((x))
alllabels = np.hstack((y))
test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))

In [143]:
# split data
x_train, x_test, y_train, y_test, test_set = split_data_equal(x=allfeatures, y=alllabels, test_set=test_set, train_size=.75)
n = x_train.shape[0]
d = x_train.shape[1]

In [144]:
# run PCA to reduce dimensionality
x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)

In [154]:
'''
Fitting various models is time intensive, especially when trying out multiple sets of features.
For this reason, the code below has been commented out and the allfeatures and alllabels are
set statically at what they were last processed as. Pickle files were used to save the models
and are called in the last cell to show results.
'''

# one-vs-one
# ovo_linSVC_estimator = LinearSVC(dual=False, fit_intercept=False)
# ovo_linSVC = OneVsOneClassifier(ovo_linSVC_estimator, n_jobs=-1)
# parameters = {'estimator__C':[10**i for i in range(-2, 2)]}
# ovo_linSVCCV = GridSearchCV(ovo_linSVC, parameters, n_jobs=-1).fit(x_train, y_train)

# one-vs-rest
# ovr_linSVC = LinearSVC(dual=False, fit_intercept=False)
# parameters = {'C':[10**i for i in range(-2, 2)]}
# ovr_linSVCCV = GridSearchCV(ovr_linSVC, parameters, n_jobs=-1).fit(x_train, y_train)

# crammer-singer
# mult_linSVC = LinearSVC(multi_class='crammer_singer', fit_intercept=False)
# parameters = {'C':[10**i for i in range(-2, 2)]}
# mult_linSVCCV = GridSearchCV(mult_linSVC, parameters, n_jobs=-1).fit(x_train, y_train)

'\nFitting various models is time intensive, especially when trying out multiple sets of features.\nFor this reason, the code below has been commented out and the allfeatures and alllabels are\nset statically at what they were last processed as. Pickle files were used to save the models\nand are called in the last cell to show results.\n'

In [155]:
# y_predict_ovo = ovo_linSVCCV.predict(x_test)
# y_predict_ovr = ovr_linSVCCV.predict(x_test)
# y_predict_mult = mult_linSVCCV.predict(x_test)
# print('Accuracy: %f%%' % (np.mean(y_predict_ovo == y_test)*100))
# print('Accuracy: %f%%' % (np.mean(y_predict_ovr == y_test)*100))
# print('Accuracy: %f%%' % (np.mean(y_predict_mult == y_test)*100))

## Aggregation of results

In [149]:
# for purposes of efficiency, the models were saved using pickle
all_test_set_predictions_header = ['Id']
all_test_set_predictions = np.arange(1, 4321)

# declare a dictionary for models
my_dict = {}

# image transformations feature sets
feature_sets = ['base',
                'color',
                'compress',
                'crop',
                'crop_to_corner',
                'homography',
                'mirror',
                'rotate30',
                'scale']

#### one-vs-one

In [152]:
# ovo - print all past results for each image transformation added to the base set
print('one-vs-one\nadding image transformations 1 at a time')
for i in feature_sets:
    my_dict['ovo_linSVCCV_{0}'.format(i)] = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/LinearSVC/ovo_linSVCCV_' + i))
    if i == 'base':
        allfeatures = x
        alllabels = y
    else:
        allfeatures = np.vstack((x, globals()[i + '_set']))
        alllabels = np.hstack((y, y))
    test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
    x_train, x_test, y_train, y_test, test_set = split_data_equal(x=allfeatures, y=alllabels, test_set=test_set, train_size=.75)
    x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)
    all_test_set_predictions_header = all_test_set_predictions_header + ['ovo_' + i]
    all_test_set_predictions = np.vstack((all_test_set_predictions, my_dict['ovo_linSVCCV_' + i].predict(test_set)))
    print(i, ': %f%%' % (np.mean(my_dict['ovo_linSVCCV_' + i].predict(x_test) == y_test)*100))

one-vs-one
adding image transformations 1 at a time
base : 70.555556%
color : 66.435185%
compress : 57.222222%
crop : 84.583333%
crop_to_corner : 65.046296%
homography : 90.601852%
mirror : 92.314815%
rotate30 : 48.055556%
scale : 68.611111%


#### one-vs-rest

In [150]:
# ovr - print all past results for each image transformation added to the base set
print('one-vs-rest\nadding image transformations 1 at a time')
for i in feature_sets:
    my_dict['ovr_linSVCCV_{0}'.format(i)] = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/LinearSVC/ovr_linSVCCV_' + i))
    if i == 'base':
        allfeatures = x
        alllabels = y
    else:
        allfeatures = np.vstack((x, globals()[i + '_set']))
        alllabels = np.hstack((y, y))
    test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
    x_train, x_test, y_train, y_test, test_set = split_data_equal(x=allfeatures, y=alllabels, test_set=test_set, train_size=.75)
    x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)
    all_test_set_predictions_header = all_test_set_predictions_header + ['ovr_' + i]
    all_test_set_predictions = np.vstack((all_test_set_predictions, my_dict['ovr_linSVCCV_' + i].predict(test_set)))
    print(i, ': %f%%' % (np.mean(my_dict['ovr_linSVCCV_' + i].predict(x_test) == y_test)*100))

one-vs-rest
adding image transformations 1 at a time
base : 57.592593%
color : 52.731481%
compress : 46.527778%
crop : 65.370370%
crop_to_corner : 51.342593%
homography : 67.361111%
mirror : 70.648148%
rotate30 : 40.185185%
scale : 52.824074%


#### multinomial

In [151]:
# multinomial - print all past results for each image transformation added to the base set
print('multinomial\nadding image transformations 1 at a time')
for i in feature_sets:
    my_dict['mult_linSVCCV_{0}'.format(i)] = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/LinearSVC/mult_linSVCCV_' + i))
    if i == 'base':
        allfeatures = x
        alllabels = y
    else:
        allfeatures = np.vstack((x, globals()[i + '_set']))
        alllabels = np.hstack((y, y))
    test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
    x_train, x_test, y_train, y_test, test_set = split_data_equal(x=allfeatures, y=alllabels, test_set=test_set, train_size=.75)
    x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)
    all_test_set_predictions_header = all_test_set_predictions_header + ['mult_' + i]
    all_test_set_predictions = np.vstack((all_test_set_predictions, my_dict['mult_linSVCCV_' + i].predict(test_set)))
    print(i, ': %f%%' % (np.mean(my_dict['mult_linSVCCV_' + i].predict(x_test) == y_test)*100))

multinomial
adding image transformations 1 at a time
base : 69.074074%
color : 65.555556%
compress : 56.944444%
crop : 82.546296%
crop_to_corner : 64.398148%
homography : 89.398148%
mirror : 91.388889%
rotate30 : 47.361111%
scale : 66.296296%


#### output all predictions

In [153]:
df = pd.DataFrame(all_test_set_predictions.astype(int).T)
df.to_csv('All_LinearSVC_Predictions.csv', sep=',', header=all_test_set_predictions_header, index=False)